In [52]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [5]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, lh, rh])


In [6]:
DATA_PATH = '/home/smayan/Desktop/ASL/dataset/SL'
sequence_length = 30
min_sequences_per_class = 10

In [13]:
actions = [
    'a', 'about', 'again', 'all', 'also', 'always', 'and', 'angry', 'animal', 'answer', 
    'apple', 'ask', 'baby', 'bad', 'bathroom', 'beautiful', 'because', 'bed', 'before', 
    'big', 'book', 'boy', 'brother', 'but', 'buy', 'bye', 'call', 'can', 'car', 'cat', 
    'city', 'class', 'clean', 'clothes', 'cold', 'college', 'color', 'come', 'computer', 
    'cook', 'dad', 'day', 'deaf', 'different', 'doctor', 'dog', 'done', "don't want", 
    'down', 'drink', 'eat', 'eight', 'enough', 'family', 'fast', 'father', 'feel', 
    'find', 'fine', 'finish', 'first', 'five', 'food', 'for', 'four', 'friend', 'from', 
    'get', 'girl', 'give', 'go', 'good', 'goodbye', 'happy', 'hard', 'have', 
    'head', 'hearing', 'hello', 'help', 'her', 'here', 'home', 'hospital', 'hot', 
    'house', 'how', 'hungry', 'i', 'if', 'in', 'know', 'language', 'last', 'later', 
    'learn', 'letter', 'like', 'little bit', 'live', 'look at', 'love', 'make', 'man', 
    'many', 'me', 'meet', 'milk', 'mom', 'money', 'month', 'more', 'morning', 'mother', 
    'movie', 'music', 'my', 'name', 'need', 'never', 'new', 'nice', 'night', 'nine', 
    'no', 'not', 'now', 'old', 'on', 'one', 'open', 'orange', 'our', 'out', 'people', 
    'phone', 'play', 'please', 'put', 'question', 'read', 'ready', 'red', 'right', 'sad', 
    'same', 'say', 'school', 'see', 'seven', 'she', 'shirt', 'shoes', 'show', 'sick', 
    'sign', 'sign language', 'sister', 'sit', 'six', 'sleep', 'slow', 'small', 'sorry', 
    'stand', 'start', 'stop', 'store', 'story', 'student', 'study', 'talk', 'teach', 
    'teacher', 'tell', 'ten', 'thank you', 'that']
# 'the', 'their', 'they', 'thing', 
#     'think', 'thirsty', 'this', 'three', 'time', 'tired', 'to', 'today', 'tomorrow', 
#     'two', 'understand', 'up', 'use', 'wait', 'walk', 'want', 'water', 'way', 
#     'we', 'wear', 'week', 'what', 'when', 'where', 'which', 'white', 'who', 'why', 
#     'will', 'with', 'woman', 'word', 'work', 'world', 'write', 'wrong', 'year', 'yellow', 
#     'yes', 'yesterday', 'you', 'your'
# ]
label_map = {label: num for num, label in enumerate(actions)}

In [14]:
len(actions)

178

In [9]:
sequences, labels = [], []

In [10]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        video_files = [f for f in os.listdir(action_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        
        if not video_files:
            print(f"Warning: No videos found for action '{action}'. Skipping.")
            continue
            
        print(f"\nProcessing action: {action} ({len(video_files)} videos)")
        action_sequences = []

        for video_file in tqdm(video_files, desc=f"Processing videos for '{action}'"):
            video_path = os.path.join(action_path, video_file)
            cap = cv.VideoCapture(video_path)
            total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            
            if total_frames < sequence_length:
                cap.release()
                continue

            stride = max(1, (total_frames - sequence_length) // 5)

            for start_frame in range(0, total_frames - sequence_length + 1, stride):
                cap.set(cv.CAP_PROP_POS_FRAMES, start_frame)
                sequence = []

                for _ in range(sequence_length):
                    ret, frame = cap.read()
                    if not ret:
                        break
                    
                    _, results = mediapipe_detection(frame, holistic)
                    
                    keypoints = extract_keypoints(results)
                    sequence.append(keypoints)

                if len(sequence) == sequence_length:
                    action_sequences.append(sequence)
            
            cap.release()

        while len(action_sequences) < min_sequences_per_class and action_sequences:
            original_seq = np.array(action_sequences[np.random.randint(0, len(action_sequences))])
            noise = np.random.normal(0, 0.01, original_seq.shape)
            augmented_seq = original_seq + noise
            action_sequences.append(augmented_seq.tolist())

        sequences.extend(action_sequences)
        labels.extend([label_map[action]] * len(action_sequences))
        print(f"Generated {len(action_sequences)} sequences for {action}")

I0000 00:00:1755606928.183619  221828 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1755606928.235241  221928 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 570.172.08), renderer: NVIDIA GeForce RTX 4070 SUPER/PCIe/SSE2



Processing action: a (5 videos)


Processing videos for 'a':   0%|          | 0/5 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1755606928.277920  221903 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755606928.298731  221909 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755606928.300417  221905 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755606928.300865  221904 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755606928.301384  221915 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling 

Generated 26 sequences for a

Processing action: about (7 videos)


Processing videos for 'about': 100%|██████████| 7/7 [00:30<00:00,  4.39s/it]


...

Processing videos for 'thank you': 100%|██████████| 7/7 [00:29<00:00,  4.19s/it]


Generated 42 sequences for thank you

Processing action: that (4 videos)


Processing videos for 'that': 100%|██████████| 4/4 [00:15<00:00,  3.96s/it]

Generated 21 sequences for that


FileNotFoundError: [Errno 2] No such file or directory: '/home/smayan/Desktop/ASL/dataset/SL/the'

In [15]:
X = np.array(sequences)
y = np.array(labels)

if X.shape[0] == 0:
    raise ValueError("No sequences were generated. Check your DATA_PATH and video files.")

print(f"\nTotal Dataset shape: X={X.shape}, y={y.shape}")


Total Dataset shape: X=(7893, 30, 1530), y=(7893,)


In [16]:
X.shape

(7893, 30, 1530)

In [17]:
X

array([[[ 0.58083397,  0.34896696, -0.02170506, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58136213,  0.34933454, -0.02152584, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58183366,  0.34964073, -0.02145821, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.58246875,  0.34982988, -0.02131816, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58242691,  0.34963927, -0.02155266, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58259797,  0.34978211, -0.02141146, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.58253586,  0.34960929, -0.02138187, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58239955,  0.34968549, -0.02127483, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.58249861,  0.34968629, -0.02131183, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.58369589,  0.35071924, -0.02138767, ...,  

In [ ]:
x

In [18]:
y.shape

(7893,)

In [19]:
np.save('/media/smayan/500GB SSD/X.npy', X)
np.save('/media/smayan/500GB SSD/y.npy', y)

In [81]:
X = np.load('/media/smayan/500GB SSD/X.npy')
y = np.load('/media/smayan/500GB SSD/y.npy')

KeyboardInterrupt: 